In [5]:
import numpy as np
import pandas as pd
from torch_geometric.data import HeteroData
import torch
import torch_geometric.transforms as T
import pickle
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import precision_score, recall_score
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GATConv, to_hetero

In [6]:

# Usage
file_path_replies = r"/home/azureuser/rumour-detection-pheme/replies_charlie_hebdo.pkl"
file_path_posts = r"/home/azureuser/rumour-detection-pheme/posts_charlie_hebdo.pkl"



In [3]:

class HeteroDataProcessorFilterNodeonTest:
    def __init__(self, file_path_replies, file_path_posts, time_cut=15):
        self.file_path_replies = file_path_replies
        self.file_path_posts = file_path_posts
        self.time_cut = time_cut
        self.df_replies = None
        self.df_posts = None

    def load_data(self):
        self.df_replies = pd.read_pickle(self.file_path_replies)
        self.df_posts = pd.read_pickle(self.file_path_posts)

    def process_data(self):
        # Define post and reply features
        post_features = ['followers', 'favorite_count', 'retweet_count', 'verified', 'rumour', 'id', 'embeddings_avg']
        reply_features = ['reply_followers', 'reply_user_id', 'reply_verified', 'time_diff', 'reply_embeddings_avg', 'id']

        # Filter and group replies
        self.df_replies['min_since_fst_post'] = round(
            (self.df_replies['time'] - self.df_replies['time'].min()).dt.total_seconds() / 60, 2)
        grouped_replies = self.df_replies.groupby(['id']).agg(
            replies=('time_diff', 'count'),
            first_time_diff=('time_diff', 'first')
        ).reset_index()

        # Merge posts and replies
        self.df_posts = self.df_posts[post_features].merge(grouped_replies, on="id", how="inner")
        self.df_posts['replies'] = self.df_posts['replies'].fillna(0)
        self.df_posts['first_time_diff'] = self.df_posts['first_time_diff'].fillna(0)

        # One-hot encoding for verified columns
        self.df_posts['verified'] = self.df_posts['verified'].astype(str).replace({'True': '1', 'False': '0'}).astype(int)
        self.df_posts = pd.concat([self.df_posts, pd.get_dummies(self.df_posts["verified"], dtype=int)], axis=1)
        self.df_posts.drop(["verified"], axis=1, inplace=True)
        self.df_posts.rename(columns={1: 'verified', 0: 'no_verified'}, inplace=True)

        self.df_replies['reply_verified'] = self.df_replies['reply_verified'].astype(str).replace({'True': '1', 'False': '0'}).astype(int)
        self.df_replies = pd.concat([self.df_replies, pd.get_dummies(self.df_replies["reply_verified"], dtype=int)], axis=1)
        self.df_replies.drop(["reply_verified"], axis=1, inplace=True)
        self.df_replies.rename(columns={1: 'reply_verified', 0: 'reply_no_verified'}, inplace=True)

        # Train/test split
        train, not_train = train_test_split(self.df_posts, test_size=0.3, random_state=42, stratify=self.df_posts['rumour'])
        val, test = train_test_split(not_train, test_size=0.5, random_state=42, stratify=not_train['rumour'])

        # Post features processing
        post_features = train[["followers", "favorite_count", "retweet_count", "no_verified", "verified", "first_time_diff"]]
        scaler_posts = RobustScaler()
        scaled_features =scaler_posts.fit_transform(post_features[['followers', 'favorite_count', 'retweet_count', 'first_time_diff']])
        scaled_data = pd.DataFrame(scaled_features, columns=['followers', 'favorite_count', 'retweet_count', 'first_time_diff'])
        scaled_data['no_verified'] = np.array(train['no_verified'])
        scaled_data['verified'] = np.array(train['verified'])
        post_features = scaled_data
        post_embeddings = np.array(train['embeddings_avg'].tolist())
        x1 = np.concatenate((post_features, post_embeddings), axis=1)

        # Reply features processing
        scaler_replies = RobustScaler()
        reply_features = self.df_replies[self.df_replies.id.isin(np.array(train.id))][["reply_followers", "reply_no_verified", "reply_verified", "time_diff"]]
        reply_features[['reply_followers', 'time_diff']] = scaler_replies.fit_transform(reply_features[['reply_followers', 'time_diff']])
        reply_embeddings = np.array(self.df_replies[self.df_replies.id.isin(np.array(train.id))]['reply_embeddings_avg'].tolist())
        x2 = np.concatenate((reply_features, reply_embeddings), axis=1)

        # Test/validation data preparation
        test_val_df_replies = pd.read_pickle(self.file_path_replies)
        test_val_df_posts = pd.read_pickle(self.file_path_posts)
        test_val_df_posts = test_val_df_posts[~test_val_df_posts.id.isin(train.id)]
        test_val_df_replies = test_val_df_replies[~test_val_df_replies.id.isin(train.id)]

        post_features = ['followers', 'favorite_count', 'retweet_count', 'verified', 'rumour', 'id', 'embeddings_avg']
        test_val_reply_features = ['reply_followers', 'reply_user_id', 'reply_verified', 'time_diff', 'reply_embeddings_avg', 'id']
        
        test_val_df_replies['min_since_fst_post'] = round((test_val_df_replies['time'] - test_val_df_replies['time'].min())\
        .dt.total_seconds() / 60,2)
        
        test_val_df_replies = test_val_df_replies[test_val_reply_features][(test_val_df_replies.time_diff <= time_cut)&\
           (test_val_df_replies.min_since_fst_post <= time_cut)]
        grouped_replies = test_val_df_replies.groupby(['id']).agg(
            replies=('time_diff', 'count'),
            first_time_diff=('time_diff', 'first')
        ).reset_index()
        
        test_val_df_posts = test_val_df_posts[post_features].merge(grouped_replies, on="id", how="inner")
        test_val_df_posts['replies'] = test_val_df_posts['replies'].fillna(0)
        test_val_df_posts['first_time_diff'] = test_val_df_posts['first_time_diff'].fillna(0)
        
        # One-hot encoding for verified columns
        test_val_df_posts['verified'] = test_val_df_posts['verified'].astype(str).replace({'True': '1', 'False': '0'}).astype(int)
        test_val_df_posts = pd.concat([test_val_df_posts, pd.get_dummies(test_val_df_posts["verified"], dtype=int)], axis=1)
        test_val_df_posts.drop(["verified"], axis=1, inplace=True)
        test_val_df_posts.rename(columns={1: 'verified', 0: 'no_verified'}, inplace=True)
        
        test_val_df_replies['reply_verified'] = test_val_df_replies['reply_verified'].astype(str).replace({'True': '1', 'False': '0'}).astype(int)
        test_val_df_replies = pd.concat([test_val_df_replies, pd.get_dummies(test_val_df_replies["reply_verified"], dtype=int)], axis=1)
        test_val_df_replies.drop(["reply_verified"], axis=1, inplace=True)
        test_val_df_replies.rename(columns={1: 'reply_verified', 0: 'reply_no_verified'}, inplace=True)
        
        test_val_df_posts = test_val_df_posts.merge(pd.concat([val,test])[['id']].reset_index(),on='id',how='left')
        test_val_df_posts.set_index('index',drop=True,inplace=True)
        
        post_features = test_val_df_posts[["followers", "favorite_count", "retweet_count", "no_verified", "verified", "first_time_diff"]]
        
        
        scaled_features = scaler_posts.transform(post_features[['followers', 'favorite_count', 'retweet_count', 'first_time_diff']])
        
        # Convert the scaled features back to a DataFrame
        scaled_data = pd.DataFrame(scaled_features, columns=['followers', 'favorite_count', 'retweet_count', 'first_time_diff'])
        
        # Add the binary features back to the scaled data
        scaled_data['no_verified'] = np.array(post_features['no_verified'])
        scaled_data['verified'] = np.array(post_features['verified'])
        post_features = scaled_data
        
        post_embeddings = np.array(test_val_df_posts['embeddings_avg'].tolist())
        #post_features = scaler.fit_transform(post_features)
        x3 = np.concatenate((post_features, post_embeddings), axis=1)
        
        test_val_reply_features =  test_val_df_replies[["reply_followers", "reply_no_verified","reply_verified","time_diff"]]
        test_val_reply_features[['reply_followers','time_diff']] = scaler_replies.transform(test_val_reply_features[['reply_followers','time_diff']])
        
        test_val_reply_embeddings = np.array(test_val_df_replies['reply_embeddings_avg'].tolist())
        x4 = np.concatenate((test_val_reply_features, test_val_reply_embeddings), axis=1)
        
        
        # Mapping post ids
        post_map = {value: i for i, value in enumerate(pd.concat([train[['id']],test_val_df_posts[['id']]])['id'].unique())}
        df_replies_edges = pd.concat([self.df_replies[self.df_replies.id.isin(np.array(train.id))][["id", "reply_user_id"]],\
                                test_val_df_replies[["id", "reply_user_id"]]])
        
        df_replies_edges["id"] = df_replies_edges['id'].map(post_map).astype(int)
        
        # Mapping reply user ids
        reply_user_map = {value: i for i, value in enumerate(df_replies_edges['reply_user_id'].unique())}
        df_replies_edges["reply_user_id"] = df_replies_edges["reply_user_id"].map(reply_user_map)

        return train,test_val_df_posts,df_replies_edges,x1,x2,x3,x4


    def create_heterodata(self,train,test_val_df_posts,df_replies_edges,x1,x2,x3,x4):

        y = pd.concat([train['rumour'],test_val_df_posts['rumour']]).to_numpy()
        edge_index = df_replies_edges.values.transpose()
        x = np.concatenate((x1,x3))
        x_reply = np.concatenate((x2,x4))
            
        num_rows = x.shape[0]
        train_mask = np.zeros(num_rows, dtype=bool)
        val_mask = np.zeros(num_rows, dtype=bool)
        test_mask = np.zeros(num_rows, dtype=bool)
        train_mask[:-x3.shape[0]]=True
        val_mask[-x3.shape[0]:-int(x3.shape[0]/2)]=True
        test_mask[-int(x3.shape[0]/2):]=True
            
        data = HeteroData()
        data['id'].x = torch.tensor(x, dtype=torch.float32)
        data['id'].y =  torch.from_numpy(y)
        data['id'].train_mask = torch.tensor(train_mask)
        data['id'].val_mask = torch.tensor(val_mask) 
        data['id'].test_mask = torch.tensor(test_mask)
        data['reply_user_id'].x = torch.tensor(x_reply, dtype=torch.float32)
        data['id', 'retweet', 'reply_user_id'].edge_index = torch.from_numpy(edge_index.reshape(2,len(x_reply)))
        data = T.ToUndirected()(data)
        
        return data

    def process(self):
            
        self.load_data()
        train,test_val_df_posts,df_replies_edges,x1,x2,x3,x4 = self.process_data()
        return self.create_heterodata(train,test_val_df_posts,df_replies_edges,x1,x2,x3,x4)

In [9]:

class HeteroDataProcessorFilterNodeonTestV2:
    #Different time cuts 
    def __init__(self, file_path_replies, file_path_posts, time_cut_replies=15,time_cut_posts=15):
        self.file_path_replies = file_path_replies
        self.file_path_posts = file_path_posts
        self.time_cut_replies = time_cut_replies
        self.time_cut_posts = time_cut_posts
        self.df_replies = None
        self.df_posts = None

    def load_data(self):
        self.df_replies = pd.read_pickle(self.file_path_replies)
        self.df_posts = pd.read_pickle(self.file_path_posts)

    def process_data(self):
        # Define post and reply features
        post_features = ['followers', 'favorite_count', 'retweet_count', 'verified', 'rumour', 'id', 'embeddings_avg']
        reply_features = ['reply_followers', 'reply_user_id', 'reply_verified', 'time_diff', 'reply_embeddings_avg', 'id']

        # Filter and group replies
        self.df_replies['min_since_fst_post'] = round(
            (self.df_replies['time'] - self.df_replies['time'].min()).dt.total_seconds() / 60, 2)
        grouped_replies = self.df_replies.groupby(['id']).agg(
            replies=('time_diff', 'count'),
            first_time_diff=('time_diff', 'first')
        ).reset_index()

        # Merge posts and replies
        self.df_posts = self.df_posts[post_features].merge(grouped_replies, on="id", how="inner")
        self.df_posts['replies'] = self.df_posts['replies'].fillna(0)
        self.df_posts['first_time_diff'] = self.df_posts['first_time_diff'].fillna(0)

        # One-hot encoding for verified columns
        self.df_posts['verified'] = self.df_posts['verified'].astype(str).replace({'True': '1', 'False': '0'}).astype(int)
        self.df_posts = pd.concat([self.df_posts, pd.get_dummies(self.df_posts["verified"], dtype=int)], axis=1)
        self.df_posts.drop(["verified"], axis=1, inplace=True)
        self.df_posts.rename(columns={1: 'verified', 0: 'no_verified'}, inplace=True)

        self.df_replies['reply_verified'] = self.df_replies['reply_verified'].astype(str).replace({'True': '1', 'False': '0'}).astype(int)
        self.df_replies = pd.concat([self.df_replies, pd.get_dummies(self.df_replies["reply_verified"], dtype=int)], axis=1)
        self.df_replies.drop(["reply_verified"], axis=1, inplace=True)
        self.df_replies.rename(columns={1: 'reply_verified', 0: 'reply_no_verified'}, inplace=True)

        # Train/test split
        train, not_train = train_test_split(self.df_posts, test_size=0.3, random_state=42, stratify=self.df_posts['rumour'])
        val, test = train_test_split(not_train, test_size=0.5, random_state=42, stratify=not_train['rumour'])

        # Post features processing
        post_features = train[["followers", "favorite_count", "retweet_count", "no_verified", "verified", "first_time_diff"]]
        scaler_posts = RobustScaler()
        scaled_features =scaler_posts.fit_transform(post_features[['followers', 'favorite_count', 'retweet_count', 'first_time_diff']])
        scaled_data = pd.DataFrame(scaled_features, columns=['followers', 'favorite_count', 'retweet_count', 'first_time_diff'])
        scaled_data['no_verified'] = np.array(train['no_verified'])
        scaled_data['verified'] = np.array(train['verified'])
        post_features = scaled_data
        post_embeddings = np.array(train['embeddings_avg'].tolist())
        x1 = np.concatenate((post_features, post_embeddings), axis=1)

        # Reply features processing
        scaler_replies = RobustScaler()
        reply_features = self.df_replies[self.df_replies.id.isin(np.array(train.id))][["reply_followers", "reply_no_verified", "reply_verified", "time_diff"]]
        reply_features[['reply_followers', 'time_diff']] = scaler_replies.fit_transform(reply_features[['reply_followers', 'time_diff']])
        reply_embeddings = np.array(self.df_replies[self.df_replies.id.isin(np.array(train.id))]['reply_embeddings_avg'].tolist())
        x2 = np.concatenate((reply_features, reply_embeddings), axis=1)

        # Test/validation data preparation
        test_val_df_replies = pd.read_pickle(self.file_path_replies)
        test_val_df_posts = pd.read_pickle(self.file_path_posts)
        test_val_df_posts = test_val_df_posts[~test_val_df_posts.id.isin(train.id)]
        test_val_df_replies = test_val_df_replies[~test_val_df_replies.id.isin(train.id)]

        post_features = ['followers', 'favorite_count', 'retweet_count', 'verified', 'rumour', 'id', 'embeddings_avg']
        test_val_reply_features = ['reply_followers', 'reply_user_id', 'reply_verified', 'time_diff', 'reply_embeddings_avg', 'id']
        
        test_val_df_replies['min_since_fst_post'] = round((test_val_df_replies['time'] - test_val_df_replies['time'].min())\
        .dt.total_seconds() / 60,2)
        
        test_val_df_replies = test_val_df_replies[test_val_reply_features][(test_val_df_replies.time_diff <= self.time_cut_replies)&\
           (test_val_df_replies.min_since_fst_post <= self.time_cut_posts)]
        grouped_replies = test_val_df_replies.groupby(['id']).agg(
            replies=('time_diff', 'count'),
            first_time_diff=('time_diff', 'first')
        ).reset_index()
        
        test_val_df_posts = test_val_df_posts[post_features].merge(grouped_replies, on="id", how="inner")
        test_val_df_posts['replies'] = test_val_df_posts['replies'].fillna(0)
        test_val_df_posts['first_time_diff'] = test_val_df_posts['first_time_diff'].fillna(0)
        
        # One-hot encoding for verified columns
        test_val_df_posts['verified'] = test_val_df_posts['verified'].astype(str).replace({'True': '1', 'False': '0'}).astype(int)
        test_val_df_posts = pd.concat([test_val_df_posts, pd.get_dummies(test_val_df_posts["verified"], dtype=int)], axis=1)
        test_val_df_posts.drop(["verified"], axis=1, inplace=True)
        test_val_df_posts.rename(columns={1: 'verified', 0: 'no_verified'}, inplace=True)
        
        test_val_df_replies['reply_verified'] = test_val_df_replies['reply_verified'].astype(str).replace({'True': '1', 'False': '0'}).astype(int)
        test_val_df_replies = pd.concat([test_val_df_replies, pd.get_dummies(test_val_df_replies["reply_verified"], dtype=int)], axis=1)
        test_val_df_replies.drop(["reply_verified"], axis=1, inplace=True)
        test_val_df_replies.rename(columns={1: 'reply_verified', 0: 'reply_no_verified'}, inplace=True)
        
        test_val_df_posts = test_val_df_posts.merge(pd.concat([val,test])[['id']].reset_index(),on='id',how='left')
        test_val_df_posts.set_index('index',drop=True,inplace=True)
        
        post_features = test_val_df_posts[["followers", "favorite_count", "retweet_count", "no_verified", "verified", "first_time_diff"]]
        
        
        scaled_features = scaler_posts.transform(post_features[['followers', 'favorite_count', 'retweet_count', 'first_time_diff']])
        
        # Convert the scaled features back to a DataFrame
        scaled_data = pd.DataFrame(scaled_features, columns=['followers', 'favorite_count', 'retweet_count', 'first_time_diff'])
        
        # Add the binary features back to the scaled data
        scaled_data['no_verified'] = np.array(post_features['no_verified'])
        scaled_data['verified'] = np.array(post_features['verified'])
        post_features = scaled_data
        
        post_embeddings = np.array(test_val_df_posts['embeddings_avg'].tolist())
        #post_features = scaler.fit_transform(post_features)
        x3 = np.concatenate((post_features, post_embeddings), axis=1)
        
        test_val_reply_features =  test_val_df_replies[["reply_followers", "reply_no_verified","reply_verified","time_diff"]]
        test_val_reply_features[['reply_followers','time_diff']] = scaler_replies.transform(test_val_reply_features[['reply_followers','time_diff']])
        
        test_val_reply_embeddings = np.array(test_val_df_replies['reply_embeddings_avg'].tolist())
        x4 = np.concatenate((test_val_reply_features, test_val_reply_embeddings), axis=1)
        
        
        # Mapping post ids
        post_map = {value: i for i, value in enumerate(pd.concat([train[['id']],test_val_df_posts[['id']]])['id'].unique())}
        df_replies_edges = pd.concat([self.df_replies[self.df_replies.id.isin(np.array(train.id))][["id", "reply_user_id"]],\
                                test_val_df_replies[["id", "reply_user_id"]]])
        
        df_replies_edges["id"] = df_replies_edges['id'].map(post_map).astype(int)
        
        # Mapping reply user ids
        reply_user_map = {value: i for i, value in enumerate(df_replies_edges['reply_user_id'].unique())}
        df_replies_edges["reply_user_id"] = df_replies_edges["reply_user_id"].map(reply_user_map)

        return train,test_val_df_posts,df_replies_edges,x1,x2,x3,x4


    def create_heterodata(self,train,test_val_df_posts,df_replies_edges,x1,x2,x3,x4):

        y = pd.concat([train['rumour'],test_val_df_posts['rumour']]).to_numpy()
        edge_index = df_replies_edges.values.transpose()
        x = np.concatenate((x1,x3))
        x_reply = np.concatenate((x2,x4))
            
        num_rows = x.shape[0]
        train_mask = np.zeros(num_rows, dtype=bool)
        val_mask = np.zeros(num_rows, dtype=bool)
        test_mask = np.zeros(num_rows, dtype=bool)
        train_mask[:-x3.shape[0]]=True
        val_mask[-x3.shape[0]:-int(x3.shape[0]/2)]=True
        test_mask[-int(x3.shape[0]/2):]=True
            
        data = HeteroData()
        data['id'].x = torch.tensor(x, dtype=torch.float32)
        data['id'].y =  torch.from_numpy(y)
        data['id'].train_mask = torch.tensor(train_mask)
        data['id'].val_mask = torch.tensor(val_mask) 
        data['id'].test_mask = torch.tensor(test_mask)
        data['reply_user_id'].x = torch.tensor(x_reply, dtype=torch.float32)
        data['id', 'retweet', 'reply_user_id'].edge_index = torch.from_numpy(edge_index.reshape(2,len(x_reply)))
        data = T.ToUndirected()(data)
        
        return data

    def process(self):
            
        self.load_data()
        train,test_val_df_posts,df_replies_edges,x1,x2,x3,x4 = self.process_data()
        return self.create_heterodata(train,test_val_df_posts,df_replies_edges,x1,x2,x3,x4)

In [10]:

# Usage
file_path_replies = r"/home/azureuser/rumour-detection-pheme/replies_charlie_hebdo.pkl"
file_path_posts = r"/home/azureuser/rumour-detection-pheme/posts_charlie_hebdo.pkl"
time_cut =12

processor = HeteroDataProcessorFilterNodeonTestV2(file_path_replies, file_path_posts, time_cut_replies=10,time_cut_posts=50)
data = processor.process()


/tmp/ipykernel_5639/450977190.py:121: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_val_reply_features[['reply_followers','time_diff']] = scaler_replies.transform(test_val_reply_features[['reply_followers','time_diff']])


In [11]:
data

HeteroData(
  id={
    x=[1430, 106],
    y=[1430],
    train_mask=[1430],
    val_mask=[1430],
    test_mask=[1430],
  },
  reply_user_id={ x=[13286, 104] },
  (id, retweet, reply_user_id)={ edge_index=[2, 13286] },
  (reply_user_id, rev_retweet, id)={ edge_index=[2, 13286] }
)

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GATConv, to_hetero

class GAT(torch.nn.Module):
    def __init__(self, dim_h, dim_out):
        super().__init__()
        self.conv1 = GATConv((-1, -1), dim_h, add_self_loops=False)
        self.conv2 = GATConv(dim_h, dim_h, add_self_loops=False)  # Added second GATConv layer
        self.linear = nn.Linear(dim_h, dim_out)
        self.dropout = nn.Dropout(p=0.4)

    def forward(self, x, edge_index):
        h = self.conv1(x, edge_index).relu()
        h = self.dropout(h)
        h = self.conv2(h, edge_index).relu()  # Pass through the second GATConv layer
        h = self.dropout(h)
        h = self.linear(h)
        return h

@torch.no_grad()
def test(mask):
    model.eval()
    pred = model(data.x_dict, data.edge_index_dict)['id'].argmax(dim=-1)
    acc = (pred[mask] == data['id'].y[mask]).sum() / mask.sum()
    return float(acc)

In [6]:
data

HeteroData(
  id={
    x=[1495, 106],
    y=[1495],
    train_mask=[1495],
    val_mask=[1495],
    test_mask=[1495],
  },
  reply_user_id={ x=[13922, 104] },
  (id, retweet, reply_user_id)={ edge_index=[2, 13922] },
  (reply_user_id, rev_retweet, id)={ edge_index=[2, 13922] }
)

In [60]:

model = GAT(dim_h=64, dim_out=2)
model = to_hetero(model, data.metadata(), aggr='sum')

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data, model = data.to(device), model.to(device)



for epoch in range(500):
    model.train()
    optimizer.zero_grad()
    out = model(data.x_dict, data.edge_index_dict)['id']
    mask = data['id'].train_mask
    loss = F.cross_entropy(out[mask], data['id'].y[mask])
    loss.backward()
    optimizer.step()
    
    if epoch % 50 == 0:
        train_acc = test(data['id'].train_mask)
        val_acc = test(data['id'].val_mask)
        print(f'Epoch: {epoch:>3} | Train Loss: {loss:.4f} | Train Acc: {train_acc*100:.2f}% | Val Acc: {val_acc*100:.2f}%')
    
test_acc = test(data['id'].test_mask)
print(f'Test accuracy: {test_acc*100:.2f}%')
    


Epoch:   0 | Train Loss: 0.7825 | Train Acc: 75.30% | Val Acc: 34.04%
Epoch:  50 | Train Loss: 0.3850 | Train Acc: 82.30% | Val Acc: 44.68%
Epoch: 100 | Train Loss: 0.3148 | Train Acc: 86.72% | Val Acc: 70.21%
Epoch: 150 | Train Loss: 0.2533 | Train Acc: 89.01% | Val Acc: 76.60%
Epoch: 200 | Train Loss: 0.1992 | Train Acc: 92.79% | Val Acc: 72.34%
Epoch: 250 | Train Loss: 0.1495 | Train Acc: 94.22% | Val Acc: 72.34%
Epoch: 300 | Train Loss: 0.1134 | Train Acc: 95.65% | Val Acc: 76.60%
Epoch: 350 | Train Loss: 0.0967 | Train Acc: 96.93% | Val Acc: 78.72%
Epoch: 400 | Train Loss: 0.0835 | Train Acc: 97.50% | Val Acc: 78.72%
Epoch: 450 | Train Loss: 0.0607 | Train Acc: 98.07% | Val Acc: 78.72%
Test accuracy: 89.36%


In [32]:
(data['id'].test_mask | data['id'].val_mask).sum()

tensor(130)

In [33]:
test_mask = data['id'].test_mask | data['id'].val_mask
pred = model(data.x_dict, data.edge_index_dict)['id'].argmax(dim=-1)
true_labels = data['id'].y[test_mask]
pred_labels = pred[test_mask]
precision_score(true_labels, pred_labels, average='macro')

0.7740384615384615

In [34]:
recall_score(true_labels, pred_labels, average='macro')

0.7314901593252108